## Machine Learning Project 2023

### Import the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.linalg import eigh
import seaborn as sns

# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
# XGBoost
import xgboost as xgb
import pandas as pd

from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_error,roc_auc_score

### Preprocess the datasets

In [ ]:
def load_datasets():
    train_dataset = np.load('fashion_train.npy')
    test_dataset = np.load('fashion_test.npy')
    return train_dataset, test_dataset

def split_datasets(train_data,test_data):
    train_X = train_data[:, :-1]
    train_y = train_data[:, -1]
    test_X = test_data[:, :-1]
    test_y = test_data[:, -1]
    return train_X, train_y, test_X, test_y

# Loading the dataset using our preprocessing script
train_dataset, test_dataset = load_datasets()

# Spliting the dataset
X_train, y_train, X_test, y_test = split_datasets(train_dataset, test_dataset)

In [ ]:
def missing_values(*dataframe):
    count = 0
    for i in dataframe:
        count += pd.DataFrame(i).isnull().sum().sum()
    if count == 0:
        return False
    else: return True

# Checking for missing values in training and test datasets
if missing_values(test_dataset) == False:
    print("There are no missing values in the test set.")
else:
    print("There are missing values in the test set.")

if missing_values(train_dataset) == False:
    print("There are no missing values in the training set.")
else:
    print("There are missing values in the training set.")

In [ ]:
# Check the dataset size
print("Training data shape:", train_dataset.shape)
print("Test data shape:", test_dataset.shape)

# Verify first few rows of training data
print(train_dataset[:5])

In [ ]:
def class_distribution(train_y, test_y):
    train, test = Counter(train_y), Counter(test_y)
    fig, ax = plt.subplots(ncols=2, figsize=(12,4))
    sns.barplot(x = ["Tshirt/Top", "Trouser","Pullover", "Dress", "Shirt"], y = list(train.values()), palette = "PuBu", ax = ax[0])
    sns.barplot(x = ["Tshirt/Top", "Trouser","Pullover", "Dress", "Shirt"], y = list(test.values()), palette = "PuBu", ax = ax[1])
    ax[0].set_title("Train split class distribution")
    ax[1].set_title("Test split class distribution")

# Check class distribution in both datasets
class_distribution(y_train, y_test)

In [ ]:
# Standardize the features
scaler = StandardScaler(with_mean=True, with_std=True)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Visualization and exploratory data analysis

#### PCA and LDA (scikit-learn)

In [ ]:
# PCA projeection using sklearn
pca = PCA(n_components=2)

X_pca = pca.fit_transform(X_train_scaled)

scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=y_train, edgecolor='k')
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Shirt']
legend1 = plt.legend(handles=scatter.legend_elements()[0], labels=class_labels, title="Clothing Types")

plt.title('PCA Projection')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.show()


In [ ]:
# LDA projeection using sklearn
lda = LinearDiscriminantAnalysis(n_components=2)

X_lda_sklearn = lda.fit_transform(X_train_scaled, y_train)

scatter = plt.scatter(X_lda_sklearn[:, 0], X_lda_sklearn[:, 1], c=y_train, edgecolor='k')
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Shirt']
legend1 = plt.legend(handles=scatter.legend_elements()[0], labels=class_labels, title="Clothing Types")

plt.title('LDA Projection')
plt.xlabel('First Linear Discriminant')
plt.ylabel('Second Linear Discriminant')
plt.show()

#### LDA implementation from scratch

In [ ]:
# LDA projection from own function
def lda_implementation(X, y):
    # Compute class mean and class scatter matrix for each class
    class_means = []
    class_scatters = []
    unique_classes = np.unique(y)

    for cls in unique_classes:
        class_data = X[y == cls]
        class_means.append(np.mean(class_data, axis=0))
        class_scatters.append(np.cov(class_data.T))

    # Sum the individual class scatter matrices
    Sw = sum(class_scatters)

    # Compute the mean
    mean = np.mean(X, axis=0)

    # Compute the between-class scatter matrix
    Sb = np.zeros((X.shape[1], X.shape[1]))
    for i in range(len(unique_classes)):
        ni = len(X[y == unique_classes[i]])
        mean_diff = class_means[i] - mean
        Sb += ni * np.outer(mean_diff, mean_diff)

    # Solve the eigenvalue problem
    eigenvalues, eigenvectors = eigh(Sb, Sw, subset_by_index=(X.shape[1] - 2, X.shape[1] - 1))


# Sort eigenvectors by descending eigenvalues
    sort_order = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, sort_order]

    # Normalize the eigenvectors
    eigenvectors_normalized = eigenvectors / np.linalg.norm(eigenvectors, axis=0)

    # Project the data onto the linear discriminant space using normalized eigenvectors
    X_lda = X.dot(eigenvectors_normalized)

    # Return transformed data and the eigenvectors
    return X_lda, eigenvectors_normalized

# Call the LDA implementation function and capture the eigenvectors
X_lda, eigenvectors_normalized = lda_implementation(X_train_scaled, y_train)

# Using the eigenvectors to transform the test data
X_lda_test = X_test_scaled.dot(eigenvectors_normalized)



### NaiveBayes Classifier

In [ ]:
def accuracy_score(y_true, y_pred):

	"""	score = (y_true - y_pred) / len(y_true) """

	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100 ,2)

def gaussian_kernel(x, data_point, bandwidth):
    return (1 / (bandwidth * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - data_point) / bandwidth) ** 2)

def kernel_density_estimation(x, data, bandwidth):
    return np.mean([gaussian_kernel(x, point, bandwidth) for point in data])

class NaiveBayes:

    def __init__(self):

        """
        Attributes:
            likelihoods: Likelihood of each feature per class
            class_priors: Prior probabilities of classes
            pred_priors: Prior probabilities of features
            features: All features of the dataset
        """
        self.features = list()
        self.likelihoods = {}
        self.class_priors = {}
        self.pred_priors = {}

        self.X_train = np.array
        self.y_train = np.array
        self.train_size = int
        self.num_feats = int

    def fit(self, X, y):

        self.features = list(range(X.shape[1]))
        self.X_train = X
        self.y_train = y
        self.train_size = X.shape[0]
        self.num_feats = X.shape[1]

        for feature in self.features:
            self.likelihoods[feature] = {}
            self.pred_priors[feature] = {}

            for feat_val in np.unique(self.X_train[:, feature]):
                self.pred_priors[feature].update({feat_val: 0})

                for outcome in np.unique(self.y_train):
                    self.likelihoods[feature].update({(feat_val, outcome): 0})
                    self.class_priors.update({outcome: 0})

        self._calc_class_prior()
        self._calc_likelihoods()
        self._calc_predictor_prior()

    def _calc_class_prior(self):
        """ P(c) - Prior Class Probability """
        for outcome in np.unique(self.y_train):
            outcome_count = np.sum(self.y_train == outcome)
            self.class_priors[outcome] = outcome_count / self.train_size

    def _calc_predictor_prior(self):
        """ P(x) - Evidence """
        for feature in self.features:
            feat_vals = dict(zip(*np.unique(self.X_train[:, feature], return_counts=True)))

            for feat_val, count in feat_vals.items():
                self.pred_priors[feature][feat_val] = count / self.train_size

    def _calc_likelihoods(self):
        for feature in self.features:
            for outcome in np.unique(self.y_train):
                outcome_data = self.X_train[self.y_train == outcome, feature]
                bandwidth = 1.06 * np.std(outcome_data) * (len(outcome_data) ** (-1/5))
                self.likelihoods[feature][outcome] = lambda x, d=outcome_data, b=bandwidth: kernel_density_estimation(x, d, b)

    def predict(self, X):
        results = []
        X = np.array(X)
        for query in X:
            probs_outcome = {}
            for outcome in np.unique(self.y_train):
                prior = np.log(self.class_priors[outcome])
                likelihood = 0  # Log likelihood of 0
                for feat_idx, feat_val in enumerate(query):
                    kde_func = self.likelihoods[feat_idx][outcome]
                    likelihood += np.log(max(kde_func(feat_val), 1e-6))  # Avoid log(0) with max()
                posterior = likelihood + prior
                probs_outcome[outcome] = posterior
            result = max(probs_outcome, key=probs_outcome.get)
            results.append(result)
        return np.array(results)

# Naive Bayes model using LDA features
nb_clf = NaiveBayes()
nb_clf.fit(X_lda, y_train)

# Predicting on LDA-transformed test data
NB_predictions = nb_clf.predict(X_lda_test)

NB_accuracy = accuracy_score(y_test, NB_predictions)
print(f"Accuracy: {NB_accuracy:.2f}")


In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, NB_predictions)
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion Matrix - Naive Bayes')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

### XGBoost Classifier

We chose the XGBoost classifier for a multi-class classification task.

The process involves hyperparameter tuning to optimize the model's performance.

Implemented from: https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/?

The `modelfit` function takes the XGBoost model, training dataset, list of feature column names and other parameters.

It performs cross-validation for hyperparameter tuning and displays model metrics and feature importances.

In [ ]:
# Define a function to fit the XGBoost model
def XGBoost(alg,                      # model instance
            dtrain,                   # training dataset
            predictors,               # list of feature column names
            useTrainCV=True,          # cross-valuation for hyperparameter tuning
            cv_folds=5,               # number of cross-validation folds
            early_stopping_rounds=50  # early stopping rounds for training
            ):
    if useTrainCV:
        # cross-validation to determine the optimal number of boosing rounds
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds,verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])

    # fit the algorithm on the data
    alg.fit(dtrain, y_train)

    # predict and evaluate on the test set
    dtrain_predictions = alg.predict(X_test_scaled)
    dtrain_predprob = alg.predict_proba(X_test_scaled)[:,1]
    dtrain_predprob_test = np.argmax(dtrain_predictions,axis=0)

    # display model report
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_test, dtrain_predictions))

    # plot feature importanace
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')


In [ ]:
# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_child_weight': [1, 2, 3, 4, 5],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'objective': ['binary:logistic', 'multi:softmax'],
    'num_class': [3, 4, 5, 6]
}

# Initialize the base model and RandomizedSearchCV
xgb_model = xgb.XGBClassifier(random_state=42)
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=50, scoring='accuracy', n_jobs=-1, cv=5, random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Print best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# Use the best estimator for further predictions
best_xgb_model = random_search.best_estimator_

# Predict and evaluate the model
final_pred_y = best_xgb_model.predict(X_test_scaled)
final_predictions = [round(value) for value in final_pred_y]

# Display final evaluation metrics
print("Final Model Accuracy:", accuracy_score(y_test, final_predictions))
print("Final Model F1 Score:", f1_score(y_test, final_pred_y, average='macro'))
print("Final Model Recall Score:", recall_score(y_test, final_pred_y, average='macro'))
print("Final Model Mean Squared Error:", mean_squared_error(y_test, final_pred_y))

# Confusion Matrix
cm = confusion_matrix(y_test, final_pred_y)
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion Matrix - XGBoost')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

### Logistic Regression Classifier

We chose the Multinomial Logistic Regression classifier for a multi-class classification task.

The process involves hyperparameter tuning to optimize the model's performance.

Implemented from #https://machinelearningmastery.com/multinomial-logistic-regression-with-python/

In [ ]:
# Multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=0.001, max_iter=5000)
# Fitting the model on the training dataset
model.fit(X_train_scaled, y_train)

# Model evaluation using cross-validation
cv = RepeatedStratifiedKFold(n_splits=5, random_state=1)
n_scores = cross_val_score(model, X_train_scaled, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

# Model performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

# Making predictions
MLR_pred = model.predict(X_test_scaled)
# Evaluation of the predictions
print("Accuracy:", accuracy_score(y_test, MLR_pred))
print(classification_report(y_test, MLR_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, MLR_pred)
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion Matrix - Logistic Regression (Multinomial)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Tuning regularization for multinomial logistic regression
penalties = [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0]
results, names = list(), list()
for C_val in penalties:
    # Defining model
    if C_val == 0.0:
        model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty=None, max_iter=10000)
    else:
        model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=C_val, max_iter=10000)
    # Evaluation the model
    scores = cross_val_score(model, X_train_scaled, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    results.append(scores)
    names.append('C=%.4f' % C_val)
    # Summarize
    print('>C=%.4f, Mean Accuracy: %.3f (%.3f)' % (C_val, np.mean(scores), np.std(scores)))

#Model performance for comparison
fig, ax = plt.subplots()
ax.boxplot(results, labels=names, showmeans=True)
ax.set_title('Model Accuracy for Different C values')
ax.set_xlabel('C Value')
ax.set_ylabel('Accuracy')
plt.show()
#https://machinelearningmastery.com/multinomial-logistic-regression-with-python/

In [ ]:
# Accuracy for each model
nb_accuracy = 72.84
xgb_accuracy = 87.24
mlr_accuracy = 83.7

# Classifier names
classifiers = ['Naive Bayes', 'XGBoost', 'Multinomial Logistic Regression']

# Corresponding accuracies
accuracies = [nb_accuracy, xgb_accuracy, mlr_accuracy]

# Bar plot
plt.figure(figsize=(10, 6))
plt.bar(classifiers, accuracies, color=['blue', 'green', 'orange'])

# Title and labels
plt.title('Comparison of Classifier Accuracies')
plt.xlabel('Classifiers')
plt.ylabel('Accuracy (%)')

# Displaying the values
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.5, f"{v:.2f}%", ha='center', va='bottom')

# Plot
plt.ylim([0, 100])
plt.show()